In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/3-2/"

/content/drive/MyDrive/Colab Notebooks/3-2


In [ ]:
#바이오 기업들 Id 불러와서 리스트로 저장
import pandas as pd

df = pd.read_parquet('./inner_bio_companyId.parquet')
indexes = list(df['companyId'])

In [ ]:
#바이오 마켓 데이터 불러오기
import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_market = pd.read_parquet('./market', filters=filters, columns=['companyId', 'pricingDate', 'priceClose', 'adjustmentFactor'])
df_market['companyId'] = list(map(int, df_market['companyId']))
df_market

In [ ]:
#바이오 재무 데이터 불러오기

import pyarrow.parquet as pq
import pandas as pd
filters = [('companyId', 'in', indexes)]
df_finance = pd.read_parquet('./financials', filters=filters, columns=['companyId', 'periodTypeName', 'fiscalQuarter', 'fiscalYear', 'dataItemName', 'dataItemValue', 'ISOCode', 'unitTypeValue'])

#데이터 처리를 위해 필요한 칼럼들을 Int 형태로 변환
df_finance['companyId'] = list(map(int, df_finance['companyId']))
df_finance['dataItemValue'] = list(map(float, df_finance['dataItemValue']))
df_finance['fiscalYear'] = list(map(int, df_finance['fiscalYear']))
df_finance['unitTypeValue'] = list(map(int, df_finance['unitTypeValue']))

#주어진 UnitTypeValue를 Value에 곱하여 단위를 맞춰줌

df_finance['dataItemValue'] = df_finance['dataItemValue'] * df_finance['unitTypeValue']
df_finance.drop(['unitTypeValue'], axis=1, inplace=True)

##39414143 기업의 데이터가 Outlier라고 판단하여 제거
df_finance = df_finance[df_finance['companyId'] != 39414143]
df_finance

In [ ]:
#재무 데이터 중 고유 아이템을 그룹으로 묶어서 분류
financials_unique = df_finance.groupby(['dataItemName', 'periodTypeName']).size().reset_index(name='freq')
financials_unique.drop('freq', axis=1, inplace=True)
financials_unique

In [ ]:
#한 고유 데이터가 분기별, 연간별로 두 개로 나누어져서 존재함
#그 중에 분기별 데이터만 사용함
quarter = financials_unique[financials_unique['periodTypeName'] == 'Quarterly']
quarter_item = list(quarter['dataItemName'])
quarter_item

In [ ]:
#재무 데이터가 존재하는 기업이 많이 없는 경우가 있어 데이터 당 기업이 500개 이상인 기업에 대해서만 분석을 진행
financial_item = []
quarter_i = quarter_item.copy()
for item in quarter_i:
  example_feature = df_finance[(df_finance['dataItemName'] == item) & (df_finance['periodTypeName'] == 'Quarterly')]
  id = len(example_feature['companyId'].unique())
  if id > 500:
    financial_item.append(item)

print(len(financial_item))

109


In [ ]:
#환율 데이터 불러오기 + 21년 ~ 22년까지 Currency별로 데이터프레임 정리
import pyarrow.parquet as pq
import pandas as pd
df_exchange = pd.read_parquet('./exchange')

df_ex = df_exchange.copy()
df_ex.sort_values('priceDate', ascending=True, inplace=True)
df_ex.set_index('priceDate', drop=True, inplace=True)

code = list(df_ex['ISOCode'].unique())
fx = pd.DataFrame()

for c in code:
  ex = df_ex[(df_ex['ISOCode'] == c)].copy()
  ex.rename(columns={'priceClose' : c}, inplace=True)
  ex.drop(['ISOCode'], axis=1, inplace=True)
  d = ex.loc['2021-09-22':'2023-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  fx = pd.concat([fx, d], axis=1)

In [ ]:
len(unique_id)

In [ ]:
#마켓 데이터로부터 기업의 주가를 시계열로 데이터로 정리
df_price = df_market.copy()
df_price['price'] = df_price['priceClose'] * df_price['adjustmentFactor']
df_price['price'] = df_price['price'].apply(lambda x: round(x, 4))
df_price.drop(['priceClose', 'adjustmentFactor'], axis=1, inplace=True)

df_price['companyId'] = list(map(int, df_price['companyId']))
unique_id = list(df_price['companyId'].unique())
market = pd.DataFrame()

#팩터 모델링을 위한 Training의 기간은 21년 9월 22일부터 22년 9월 22일까지
for id in unique_id:
  ex = df_price[(df_price['companyId'] == id)].copy()
  ex.rename(columns={'price' : id}, inplace=True)
  ex.drop(['companyId'], axis=1, inplace=True)
  ex.sort_values(by='pricingDate', inplace=True)
  ex.set_index('pricingDate', inplace=True, drop=True)
  d = ex.loc['2021-09-22':'2022-09-22'].copy()
  d = d[~d.index.duplicated(keep='first')]
  d = d.apply(pd.to_numeric, errors='coerce')
  market = pd.concat([market, d], axis=1)

market2 = market.copy()
mark_date = market2.isnull().sum(axis=1).index
mark_col = market2.isnull().sum(axis=0).index

#한 날짜에 850개 초과의 기업이 주가가 없다면 해당 날짜 제거 (공휴일이라 판단)
for i in mark_date:
  if market2.isnull().sum(axis=1)[i] > 850:
    market2.drop(i, inplace=True)

#한 기업이 40일 이상의 주가가 없으면 제거 (결측치가 너무 많으면 평균이나 선형보간법으로 결측치를 채우는게 무리가 있다고 판단)
for i in mark_col:
  if market2.isnull().sum(axis=0)[i] > 40:
    market2.drop(i, axis=1, inplace=True)

#선형 보간법으로 우선 채우고 연속된 결측치의 경우에는 전 값으로 결측치를 처리
market2.interpolate(method='linear', inplace=True)
market2.fillna(method='bfill', inplace=True)
market2.drop(39414143, axis=1, inplace=True)

In [ ]:
#한 재무 데이터에 대해 크기 순으로 정렬하여 상위 50%, 하위 50%를 기준으로 롱숏 포트폴리오를 구성 후 수익률 산출
#재무 데이터는 분기별로 나타나기 때문에 예를 들어 2분기 재무 데이터를 통해 3분기 포트폴리오를 구성하는 방향
#따라서 21년 2분기 데이터를 통해 21년 9월 22일 ~ 21년 9월 30일까지의 포트폴리오를 구성
#21년 3분기 데이터를 통해 21년 10월 1일 ~ 21년 12월 31일까지의 포트폴리오를 구성, 이렇게 22년 9월 22일까지 진행
def calculate(item, year, quarter):
  market3 = market2.copy()
  #아이템이랑 연도 받아서 데이터프레임 생성
  example_feature = df_finance[(df_finance['dataItemName'] == item)  & (df_finance['fiscalYear'] == year) & (df_finance['fiscalQuarter'] == quarter) & (df_finance['periodTypeName'] == 'Quarterly')]
  example = example_feature[['companyId', 'dataItemValue', 'ISOCode']].copy()

  #같은 시점에 똑같은 기업이 똑같은 재무 데이터에 대해 다른 값을 가지고 있다면 평균 내고 하나의 값으로 변환
  #ex) 21년에 A 기업의 영업 이익이 다른 값으로 2~3개가 있음 -> 평균 내서 하나만 남기기

  example2 = example.copy()
  for id in list(map(int, example2['companyId'])):
    num = len(example2[example2['companyId']==id]['dataItemValue'].values)
    if num > 1:
      idx = list(example2[example2['companyId']==id]['dataItemValue'].index)
      avg = []
      for i in idx:
        a = example2.loc[i]['dataItemValue']
        avg.append(a)
      average = sum(avg) / len(avg)
      example2.loc[idx[0]]['dataItemValue'] = average
      example2.drop([i for i in idx[0:]], inplace=True)
  example2.reset_index(drop=True, inplace=True)

  #재무 데이터가 어느 나라의 기업이냐에 따라 명시되어 있는 Currency가 다름 (ISOCode를 통해 확인 가능)
  #21년 2분기 재무 데이터를 활용하여 21년 9월 22일부터 ~ 21년 9월 30일까지의 포트폴리오를 구성하기 때문에,
  #달러로 표시되어 있는게 아니라면 21년 9월 22일의 환율을 기준으로 값을 달러로 변경
  #이런 식으로 각 포트폴리오 구성이 시작되는 날짜를 기준으로 환율을 곱하여 값을 맞춰줌

  if (year == 2021) & (quarter == 2):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-09-22", example2['ISOCode'][i]])
  elif (year == 2021) & (quarter == 3):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2021-10-01", example2['ISOCode'][i]])
  elif (year == 2021) & (quarter == 4):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-01-01", example2['ISOCode'][i]])
  elif (year == 2022) & (quarter == 1):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-04-01", example2['ISOCode'][i]])
  elif (year == 2022) & (quarter == 2):
    for i in range(len(example2)):
      if example2['ISOCode'][i] != "USD":
        example2['dataItemValue'][i] = (example2['dataItemValue'][i]) / (fx.loc["2022-07-01", example2['ISOCode'][i]])

  #작업하는 데이터의 값을 내림차순으로 정렬
  example2.drop('ISOCode', axis=1, inplace=True)
  example2.sort_values(by='dataItemValue', ascending=False, inplace=True)

  #데이터가 있는 기업들의 주가를 불러옴
  comp_id = list(map(int, example2['companyId']))
  market3 = market3.reindex(columns=comp_id)
  market3 = market3.dropna(axis=1)

  #년도에 따라서 포트폴리오를 구성하는 기간이 다르기 때문에 그 구간에서의 주가를 가져옴
  if (year == 2021) & (quarter == 2):
    price_y = market3.loc['2021-09-22':'2021-09-30'].copy()
  elif (year == 2021) & (quarter == 3):
    price_y = market3.loc['2022-10-01':'2022-12-31'].copy()
  elif (year == 2021) & (quarter == 4):
    price_y = market3.loc['2022-01-01':'2022-03-31'].copy()
  elif (year == 2022) & (quarter == 1):
    price_y = market3.loc['2022-04-01':'2022-06-30'].copy()
  elif (year == 2022) & (quarter == 2):
    price_y = market3.loc['2022-07-01':'2022-09-30'].copy()

  for col in price_y.columns:
    price_y[col] = price_y[col].apply(lambda x: float(x))


  #주가를 수익률로 변환 후에 크기 순으로 정렬되어 있기 때문에 상위 50퍼는 롱,
  #하위 50퍼는 숏을 갖는 포트폴리오의 수익률을 새로운 칼럼으로 생성
  price_y = price_y.pct_change(1).dropna()
  price_y = price_y + 1
  cum_ret = price_y.cumprod() - 1
  half = int(len(cum_ret.columns) / 2)
  cum_ret[item] = (cum_ret.iloc[:, 0:half+1].sum(axis=1) - cum_ret.iloc[:, half+1:].sum(axis=1)) / len(cum_ret.columns)

  return cum_ret[item]

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import statsmodels.api as sm

#21년 9월 22일부터 ~ 22년 9월 22일까지 각 예상치에 대해 분기별로 롱숏 포트폴리오를 구성 후 일별 수익률을 계산

pd.options.mode.chained_assignment = None
pair = [[2021, 3], [2021, 4], [2022, 1], [2022, 2]]
multi_agg = pd.DataFrame()
for item in financial_item:
  pf = calculate(item, 2021, 2)
  for y,q in pair:
      temp = calculate(item, y, q)
      pf = pd.concat([pf, temp], axis=0)

  multi_agg = pd.concat([multi_agg, pf], axis=1)

In [ ]:
multi_agg

In [ ]:
#데이터프레임 생성 과정에서 결측치 존재하면 칼럼 제거
port = multi_agg.copy()
columns_with_missing_values = list(port.columns[port.isnull().any()])
port.drop(columns_with_missing_values, axis=1, inplace=True)
port

In [ ]:
#주가 데이터로부터 수익률을 구하고 첫쨰날을 제거
market4 = market2.copy()
market4 = market4.pct_change(1)
market4.drop('2021-09-22', inplace=True)

#주가 수익률과 포트폴리오 수익률을 합쳐주며 특정 날짜에 Null 값이 존재하면 드랍
financial_final = pd.concat([market4, port], axis=1)
a = financial_final.index[financial_final.isnull().any(axis=1)]
financial_final.drop(a, inplace=True)
financial_final

In [ ]:
financial_final.to_csv('financial_final.csv', index=True)

In [ ]:
from tqdm import tqdm

In [ ]:
#Total은 바이오 섹터에 속하는 개별 기업의 개수
total = len(financial_final.columns[:-109])
#Port_col은 각 포트폴리오의 이름을 의미
port_col = list(port.columns)
count_x1 = []
#개별 기업의 수익률을 y로 두고, 각 데이터에 대한 포트폴리오를 X로 두고 1158개에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in port_col:
  cnt = 0
  for col in tqdm(financial_final.columns[:-109]):
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  count_x1.append(cnt)

In [ ]:
custom_percentiles = [0.25, 0.5, 0.75, 0.90]
pd.DataFrame(count_x1).describe(percentiles=custom_percentiles)

In [ ]:
#Total은 바이오 섹터에 속하는 개별 기업의 개수
total = len(financial_final.columns[:-109])
#Port_col은 각 포트폴리오의 이름을 의미
port_col = list(port.columns)
good_x1 = []
#개별 기업의 수익률을 y로 두고, 각 데이터에 대한 포트폴리오를 X로 두고 1158개에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in tqdm(port_col):
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  if cnt > 63:
    good_x1.append(p)

In [ ]:
pairs = [[good_x1[i], good_x1[j]] for i in range(len(good_x1)) for j in range(i + 1, len(good_x1))]
total = len(financial_final.columns[:-109])
count_x2 = []
#개별 기업의 수익률을 y로 두고, X1, X2와 1158개의 y에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in tqdm(pairs):
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)
  count_x2.append(cnt)

In [ ]:
custom_percentiles = [0.75, 0.8, 0.85, 0.90]
pd.DataFrame(count_x2).describe(percentiles=custom_percentiles)

In [ ]:
total = len(financial_final.columns[:-109])
#개별 기업의 수익률을 y로 두고, X1, X2와 1158개의 y에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
good_x2 = []
for p in tqdm(pairs):
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  if cnt >= 145:
    good_x2.append(p)

In [ ]:
good_x2

In [ ]:
final = []
for p in port_col:
  if p not in ['Long-term Investments', 'Unearned Revenue Current, Total']:
    final.append(['Long-term Investments', 'Unearned Revenue Current, Total', p])

In [ ]:
total = len(financial_final.columns[:-109])
#개별 기업의 수익률을 y로 두고, X1, X2와 1158개의 y에 대해 OLS를 돌리고,
#그 중 P-value의 값이 0.05 이하인 기업의 개수를 계산
for p in tqdm(final):
  cnt = 0
  for col in financial_final.columns[:-109]:
    res = sm.OLS(endog=financial_final[col], exog=sm.add_constant(financial_final[p])).fit()
    if res.pvalues[1] <= 0.05:
      cnt += 1
  print(p, cnt)